<a href="https://colab.research.google.com/github/AshwinDeshpande96/Measuring_the_Impact_of_Verbal_Disfluency_Tags_on_Automated_Dementia_Detection/blob/master/FineTuneLOOCV_2020_GOLD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Research/ADReSSo
# %cd /content/drive/My\ Drive/Admission/UIC/Research/Python/Dementia-Detection/Disfluency-Detection/english-fisher-annotations
!pwd

Mounted at /content/drive
/content/drive/My Drive/Research/ADReSSo
/content/drive/My Drive/Research/ADReSSo


In [ ]:
# models = ['Bert', 'Roberta', 'DistilBERT']
# _ = [print(f"[{i + 1}] {m}") for i, m in enumerate(models)]
# model_num = int(input("Choose model: ")) - 1
# if model_num not in range(len(models)):
#     raise Exception("Incorrect model chosen.")

# model_name = models[model_num]
#####################################################################
import random
import re
import csv
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score
from google.colab import drive
import datetime
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import LeaveOneOut
import time
import itertools
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.model_selection import KFold
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
# drive.mount('/content/drive')
from sklearn.metrics import precision_recall_fscore_support
from transformers import BertTokenizer
#################### Global Variables ####################

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
UNUSED_TOKEN = {1: '[unused0]',
                2: '[unused1]',
                3: '[unused2]'}
to_categorical = {'Control': 0,
                  'Dementia': 1}

################### Train and Test Sets ####################

# train_text, temp_text, train_labels, temp_labels = train_test_split(df['transcript'], df['dx'],
#                                                                     random_state=seed_val,
#                                                                     test_size=0.3,
#                                                                     stratify=df['dx'])


#################### BERT features ####################


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def get_bert_tokenizer():
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_basic_tokenize=False)
    tokenizer.add_special_tokens({ "additional_special_tokens": [ "[unused0]" ] })
    return tokenizer

def get_roberta_tokenizer():
    from transformers import RobertaTokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_basic_tokenize=False)
    tokenizer.add_special_tokens({ "additional_special_tokens": [ "[unused0]" ] })
    return tokenizer

def get_distilbert_tokenizer():
    from transformers import DistilBertTokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_basic_tokenize=False)
    tokenizer.add_special_tokens({ "additional_special_tokens": [ "[unused0]" ] })
    return tokenizer

def encode_sentence(transcript, tokenizer):
    transcript = re.sub(r"\.\s", " [SEP]", transcript)
    tokens = list(tokenizer.tokenize(transcript))
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    return token_ids


def add_padding(input_word_ids, max_seq_len):
    input_type = []
    for idx, embedding in enumerate(input_word_ids):
        embedding_len = len(embedding)
        e_input_type = np.ones(embedding_len, dtype=np.int64).tolist()
        if embedding_len < max_seq_len:
            zeros = np.zeros(max_seq_len - embedding_len, dtype=np.int64).tolist()
            e_input_type += zeros
            embedding += zeros
        elif embedding_len > max_seq_len:
            embedding = embedding[:max_seq_len - 1] + [102]
            e_input_type = e_input_type[:max_seq_len]
        input_type.append(torch.tensor([e_input_type]))
        input_word_ids[idx] = torch.tensor([embedding])
    return {'input_ids': input_word_ids, 'attention_mask': input_type}


def bert_encode(transcripts, tokenizer, max_seq_len):
    input_word_ids = [encode_sentence(s, tokenizer)
                      for s in transcripts]
    input_word_ids = add_padding(input_word_ids, max_seq_len)
    return input_word_ids
############################# Models ##################################
def get_bert_model():
    from transformers import BertForSequenceClassification
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab.
        num_labels=2,  # The number of output labels--2 for binary classification.
        # You can increase this for multi-class tasks.
        output_attentions=False,  # Whether the model returns attentions weights.
        output_hidden_states=False,  # Whether the model returns all hidden-states.
    )
    model.cuda()
    return model


def get_roberta_model():
    from transformers import RobertaForSequenceClassification
    model = RobertaForSequenceClassification.from_pretrained(
        "roberta-base",  # Use the 12-layer BERT model, with an uncased vocab.
        num_labels=2,  # The number of output labels--2 for binary classification.
        # You can increase this for multi-class tasks.
        output_attentions=False,  # Whether the model returns attentions weights.
        output_hidden_states=False,  # Whether the model returns all hidden-states.
    )
    model.cuda()
    return model

def get_distilbert_model():
    from transformers import DistilBertForSequenceClassification
    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab.
        num_labels=2,  # The number of output labels--2 for binary classification.
        # You can increase this for multi-class tasks.
        output_attentions=False,  # Whether the model returns attentions weights.
        output_hidden_states=False,  # Whether the model returns all hidden-states.
    )
    model.cuda()
    return model


In [ ]:
def reps():
  df = pd.read_pickle('data2020fisher_reps.pickle')
  X1 = df.transcript_without_tags.to_numpy()
  X2 = df.transcript_with_stopword_rep.to_numpy()
  X3 = df.transcript_with_non_stopword_rep.to_numpy()
  X4 = df.transcript_with_no_rep.to_numpy()
  X5 = df.transcript_with_no_filled_pauses.to_numpy()
  experiments = ['transcript_without_tags',
                  'transcript_with_stopword_rep',
                  'transcript_with_non_stopword_rep',
                  'transcript_with_no_rep',
                  'transcript_with_no_filled_pauses']
  Xs = [X1, X2, X3, X4, X5]
  df.dx = df.dx.apply(lambda x: to_categorical[x])
  y = df.dx.to_numpy()
  return df, experiments, Xs, y

def all_exp():
  df = pd.read_pickle('data2020fisher_all.pickle')
  experiments = ['transcript_without_tags',
                 'transcript_without_underscore',
                 'transcript_without_repetition',
                 'transcript_without_retracing',
                 'transcript_without_disfluency',
                ]
  X1 = df.transcript_without_tags.to_numpy()
  X2 = df.transcript_without_underscore.to_numpy()
  X3 = df.transcript_without_repetition.to_numpy()
  X4 = df.transcript_without_retracing.to_numpy()
  X5 = df.transcript_without_disfluency.to_numpy()
  Xs = [X1, X2, X3, X4, X5]
  df.dx = df.dx.apply(lambda x: to_categorical[x])
  y = df.dx.to_numpy()
  return df, experiments, Xs, y
def new_data():
    df = pd.read_pickle('new_data.pickle').dropna()
    df = df[df.both_eremoved != "placeholder"]
    experiments = ['all_errors', "e_tagged", "rep_eremoved", "ret_eremoved", "both_eremoved"]
    df.dx = df.dx.apply(lambda x: to_categorical[x])
    X1 = df.all_errors.to_numpy()
    X2 = df.e_tagged.to_numpy()
    X3 = df.rep_eremoved.to_numpy()
    X4 = df.ret_eremoved.to_numpy()
    X5 = df.both_eremoved.to_numpy()
    Xs = [X1, X2, X3, X4, X5]
    y = df.dx.to_numpy()
    return df, experiments, Xs, y

In [ ]:
df, experiments, Xs, y = new_data()
print(df.shape)
df.sample(10)

(1138, 15)


,utt_id,dx,mmse,disfluency_asr,transcript_with_tags,transcript_tokens,both_errors,both_etype,both_reference_text,asr_text_map,rep_eremoved,all_errors,e_tagged,ret_eremoved,both_eremoved
501,S051_0_2560,0,29.0,we _ 'll _ start _ with _ a _ girl _,*par we 'll start with the girl . 0 2560,"[*par, we, 'll, start, with, the, girl, ., 0,...","{'[/]': [], '[//]': []}","{'we_1': None, ''ll_1': None, 'start_1': None,...",we 'll start with the girl,"[(we, _, we, None), ('ll, _, 'll, None), (star...",we 'll start with a girl,we 'll start with a girl,we 'll start with a girl,we 'll start with a girl,we 'll start with a girl
855,S087_65133_74985,1,7.0,that _ 's _ all _ there _ is _,*par (.) the father is n't comin(g) . 65133 7...,"[*par, (.), the, father, is, n't, comin(g), .,...","{'[/]': [], '[//]': []}","{'the_1': None, 'father_1': None, 'is_1': None...",the father is n't coming,"[(that, _, the, None), ('s, _, father, None), ...",that 's all there is,that 's all there is,that 's all there is,that 's all there is,that 's all there is
646,S064_22479_25655,0,29.0,and _ the _ windows _ up _,*par and &uh the window 's up . 22479 25655,"[*par, and, &uh, the, window, 's, up, ., 2247...","{'[/]': [], '[//]': []}","{'and_1': None, 'uh_1': None, 'the_1': None, '...",and uh the window 's up,"[(and, _, and, None), (**, None, uh, None), (t...",and the windows up,and the windows up,and the windows up,and the windows up,and the windows up
1095,S108_100964_107229,1,13.0,existing _ here _ is _ turning _ over _,*par well this thing here is &uh turnin(g) ove...,"[*par, well, this, thing, here, is, &uh, turni...","{'[/]': [], '[//]': []}","{'well_1': None, 'this_1': None, 'thing_1': No...",well this thing here is uh turning over,"[(****, None, well, None), (****, None, this, ...",existing here is turning over,existing here is turning over,existing here is turning over,existing here is turning over,existing here is turning over
426,S039_36199_38888,0,28.0,and _ the _ sink _ is _ running _,*par and the &um &uh sink is running over . 3...,"[*par, and, the, &um, &uh, sink, is, running, ...","{'[/]': [], '[//]': []}","{'and_1': None, 'the_1': None, 'um_1': None, '...",and the um uh sink is running over,"[(and, _, and, None), (the, _, the, None), (**...",and the sink is running,and the sink is running,and the sink is running,and the sink is running,and the sink is running
1135,S110_21121_23565,1,16.0,and _ a _ stool _ is _ turned _ over _,*par and a stool is turned over . 21121 23565,"[*par, and, a, stool, is, turned, over, ., 21...","{'[/]': [], '[//]': []}","{'and_1': None, 'a_1': None, 'stool_1': None, ...",and a stool is turned over,"[(and, _, and, None), (a, _, a, None), (stool,...",and a stool is turned over,and a stool is turned over,and a stool is turned over,and a stool is turned over,and a stool is turned over
449,S041_22863_30896,0,30.0,and _ her _ right _ hand _ looks _ like _ she ...,*par and her right hand looks like she 's almo...,"[*par, and, her, right, hand, looks, like, she...","{'[/]': [], '[//]': []}","{'and_1': None, 'her_1': None, 'right_1': None...",and her right hand looks like she 's almost uh...,"[(and, _, and, None), (her, _, her, None), (ri...",and her right hand looks like she 's almost tr...,and her right hand looks like she 's almost tr...,and her right hand looks like she 's almost tr...,and her right hand looks like she 's almost tr...,and her right hand looks like she 's almost tr...
908,S092_51969_53216,1,15.0,no _ that _ 's _ all _,*par no that 's all . [ exc] 51969 53216,"[*par, no, that, 's, all, ., [, exc], 51969, ...","{'[/]': [], '[//]': []}","{'no_1': None, 'that_1': None, ''s_1': None, '...",no that 's all,"[(no, _, no, None), (that, _, that, None), ('s...",no that 's all,no that 's all,no that 's all,no that 's all,no that 's all
161,S013_22022_25481,0,30.0,you _ want _ like _ the _ windows _ open _ tha...,*par &uh do [/] do you want like the window 's...,"[*par, &uh, DOREP, [/], do, you, want, like, t..

In [ ]:
model_name = "bert"
lr = 4e-5
max_seq_len = 256
epochs = 8
# epochs = 1
batch_size_tr = 16
batch_size_ts = 32
from collections import OrderedDict
# experiments = ['transcript_without_tags', 'transcript_without_retracing','transcript_without_repetition', 'transcript_without_disfluency']
accuracy_dict = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
f1_dict = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
# class1_dict_acc = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
class1_dict_f1 = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
class1_dict_pr = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
class1_dict_re = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))

# class0_dict_acc = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
class0_dict_f1 = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
class0_dict_pr = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
class0_dict_re = OrderedDict(zip(experiments, [[] for _ in range(len(experiments))]))
loo = LeaveOneOut()


# print(f"SEED: {seed_val}\nMODEL: {model_name}\nBATCH SIZE TR: {batch_size_tr}\nBATCH SIZE TS: {batch_size_ts}\nLEARNING RATE: {lr}\nEMBEDDING LEN: {max_seq_len}\nEPOCH: {epochs}")
for seed_val in [0, 42, 23, 17, 19, 111]:
    for name, X in zip(experiments, Xs):
        print("#"*10, name)
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        kf = KFold(n_splits=5, random_state=seed_val, shuffle=True)
        total_preds = []
        total_target = []
        f1_set = []
        acc_set = []
        fold=1
        for train_index, test_index in kf.split(X):
            print('fold %d'%(fold))
            random.seed(seed_val)
            np.random.seed(seed_val)
            torch.manual_seed(seed_val)
            torch.cuda.manual_seed_all(seed_val)
            tokenizer = locals()[f"get_{model_name.lower()}_tokenizer"]()
            model = locals()[f"get_{model_name.lower()}_model"]()
            train_text, temp_text = X[train_index], X[test_index]
            train_labels, temp_labels = y[train_index], y[test_index]
            fold+=1
            
            tokens_train = bert_encode(train_text.tolist(), tokenizer, max_seq_len)
            tokens_test = bert_encode(temp_text.tolist(), tokenizer, max_seq_len)

            #################### Torch Dataset ####################

            input_ids = torch.cat(tokens_train['input_ids'], dim=0)
            attention_masks = torch.cat(tokens_train['attention_mask'], dim=0)
            labels = torch.tensor(train_labels.tolist())

            input_ids_test = torch.cat(tokens_test['input_ids'], dim=0)
            attention_masks_test = torch.cat(tokens_test['attention_mask'], dim=0)
            labels_test = torch.tensor(temp_labels.tolist())


            ####################
            dataset = TensorDataset(input_ids, attention_masks, labels)
            train_size = int(0.9 * len(dataset))
            val_size = len(dataset) - train_size
            train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
            ####################
            train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler=RandomSampler(train_dataset),  # Select batches randomly
                batch_size=batch_size_tr  # Trains with this batch size.
            )

            validation_dataloader = DataLoader(
                val_dataset,  # The validation samples.
                sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
                batch_size=batch_size_tr  # Evaluate with this batch size.
            )
            ###############################################################################

            total_steps = len(train_dataloader) * epochs

            #################### Training stuff ####################
            optimizer = AdamW(model.parameters(),
                            lr=lr,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
                            eps=1e-8  # args.adam_epsilon  - default is 1e-8.
                            )

            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=0,  # Default value in run_glue.py
                                                        num_training_steps=total_steps)




            training_stats = []

            # Measure the total training time for the whole run.
            total_t0 = time.time()

            # For each epoch...
            for epoch_i in range(0, epochs):

                # ========================================
                #               Training
                # ========================================

                # Perform one full pass over the training set.

                # print("")
                # print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
                # print('Training...')

                # Measure how long the training epoch takes.
                t0 = time.time()

                # Reset the total loss for this epoch.
                total_train_loss = 0

                # Put the model into training mode. Don't be mislead--the call to
                # `train` just changes the *mode*, it doesn't *perform* the training.
                # `dropout` and `batchnorm` layers behave differently during training
                # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
                model.train()

                # For each batch of training data...
                for step, batch in enumerate(train_dataloader):

                    # Progress update every 40 batches.
                    if step % 40 == 0 and not step == 0:
                        # Calculate elapsed time in minutes.
                        elapsed = format_time(time.time() - t0)

                        # Report progress.
                        # print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

                    # Unpack this training batch from our dataloader.
                    #
                    # As we unpack the batch, we'll also copy each tensor to the GPU using the
                    # `to` method.
                    #
                    # `batch` contains three pytorch tensors:
                    #   [0]: input ids
                    #   [1]: attention masks
                    #   [2]: labels
                    b_input_ids = batch[0].to(device)
                    b_input_mask = batch[1].to(device)
                    b_labels = batch[2].to(device)

                    # Always clear any previously calculated gradients before performing a
                    # backward pass. PyTorch doesn't do this automatically because
                    # accumulating the gradients is "convenient while training RNNs".
                    # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                    model.zero_grad()

                    # Perform a forward pass (evaluate the model on this training batch).
                    # In PyTorch, calling `model` will in turn call the model's `forward`
                    # function and pass down the arguments. The `forward` function is
                    # documented here:
                    # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
                    # The results are returned in a results object, documented here:
                    # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
                    # Specifically, we'll get the loss (because we provided labels) and the
                    # "logits"--the model outputs prior to activation.
                    try:
                        result = model(b_input_ids,
                                    token_type_ids=None,
                                    attention_mask=b_input_mask,
                                    labels=b_labels,
                                    return_dict=True)
                    except:
                        result = model(b_input_ids,
                                        #    token_type_ids=None,
                                        attention_mask=b_input_mask,
                                        labels=b_labels,
                                        return_dict=True)                    

                    loss = result.loss
                    logits = result.logits

                    # Accumulate the training loss over all of the batches so that we can
                    # calculate the average loss at the end. `loss` is a Tensor containing a
                    # single value; the `.item()` function just returns the Python value
                    # from the tensor.
                    total_train_loss += loss.item()

                    # Perform a backward pass to calculate the gradients.
                    loss.backward()

                    # Clip the norm of the gradients to 1.0.
                    # This is to help prevent the "exploding gradients" problem.
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                    # Update parameters and take a step using the computed gradient.
                    # The optimizer dictates the "update rule"--how the parameters are
                    # modified based on their gradients, the learning rate, etc.
                    optimizer.step()

                    # Update the learning rate.
                    scheduler.step()

                # Calculate the average loss over all of the batches.
                avg_train_loss = total_train_loss / len(train_dataloader)

                # Measure how long this epoch took.
                training_time = format_time(time.time() - t0)

                # print("")
                # print("  Average training loss: {0:.2f}".format(avg_train_loss))
                # print("  Training epcoh took: {:}".format(training_time))

                # ========================================
                #               Validation
                # ========================================
                # After the completion of each training epoch, measure our performance on
                # our validation set.

                # print("")
                # print("Running Validation...")

                t0 = time.time()

                # Put the model in evaluation mode--the dropout layers behave differently
                # during evaluation.
                model.eval()

                # Tracking variables
                total_eval_accuracy = 0
                total_eval_loss = 0
                nb_eval_steps = 0

                # Evaluate data for one epoch
                for batch in validation_dataloader:
                    # Unpack this training batch from our dataloader.
                    #
                    # As we unpack the batch, we'll also copy each tensor to the GPU using
                    # the `to` method.
                    #
                    # `batch` contains three pytorch tensors:
                    #   [0]: input ids
                    #   [1]: attention masks
                    #   [2]: labels
                    b_input_ids = batch[0].to(device)
                    b_input_mask = batch[1].to(device)
                    b_labels = batch[2].to(device)

                    # Tell pytorch not to bother with constructing the compute graph during
                    # the forward pass, since this is only needed for backprop (training).
                    with torch.no_grad():
                        # Forward pass, calculate logit predictions.
                        # token_type_ids is the same as the "segment ids", which
                        # differentiates sentence 1 and 2 in 2-sentence tasks.
                        result = model(b_input_ids,
                                    #    token_type_ids=None,
                                    attention_mask=b_input_mask,
                                    labels=b_labels,
                                    return_dict=True)

                    # Get the loss and "logits" output by the model. The "logits" are the
                    # output values prior to applying an activation function like the
                    # softmax.
                    loss = result.loss
                    logits = result.logits

                    # Accumulate the validation loss.
                    total_eval_loss += loss.item()

                    # Move logits and labels to CPU
                    logits = logits.detach().cpu().numpy()
                    label_ids = b_labels.to('cpu').numpy()

                    # Calculate the accuracy for this batch of test sentences, and
                    # accumulate it over all batches.
                    total_eval_accuracy += flat_accuracy(logits, label_ids)

                # Report the final accuracy for this validation run.
                avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
                # print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

                # Calculate the average loss over all of the batches.
                avg_val_loss = total_eval_loss / len(validation_dataloader)

                # Measure how long the validation run took.
                validation_time = format_time(time.time() - t0)

                # print("  Validation Loss: {0:.2f}".format(avg_val_loss))
                # print("  Validation took: {:}".format(validation_time))

                # Record all statistics from this epoch.
                training_stats.append(
                    {
                        'epoch': epoch_i + 1,
                        'Training Loss': avg_train_loss,
                        'Valid. Loss': avg_val_loss,
                        'Valid. Accur.': avg_val_accuracy,
                        'Training Time': training_time,
                        'Validation Time': validation_time
                    }
                )

            # print("")
            # print("Training complete!")

            # print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))

            # Create the DataLoader.
            prediction_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)
            prediction_sampler = SequentialSampler(prediction_data)
            prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size_ts)
            ################################################################################
            # Prediction on test set

            # print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

            # Put model in evaluation mode
            model.eval()

            # Tracking variables 
            predictions , true_labels = [], []

            # Predict 
            for batch in prediction_dataloader:
                # Add batch to GPU
                batch = tuple(t.to(device) for t in batch)
            
                # Unpack the inputs from our dataloader
                b_input_ids, b_input_mask, b_labels = batch
                
                # Telling the model not to compute or store gradients, saving memory and 
                # speeding up prediction
                with torch.no_grad():

                    # Forward pass, calculate logit predictions.
                    try:
                        result = model(b_input_ids, 
                                        token_type_ids=None, 
                                        attention_mask=b_input_mask,
                                        return_dict=True)
                    except:
                        result = model(b_input_ids, 
                                        #  token_type_ids=None, 
                                        attention_mask=b_input_mask,
                                        return_dict=True)

                logits = result.logits

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                
                # Store predictions and true labels
                predictions.append(logits)
                true_labels.append(label_ids)

            # print('    DONE.')
            ###############################################################################
            
            

            # Evaluate each test batch using Matthew's correlation coefficient
            # print('Calculating Acc and F1 Corr. Coef. for each batch...')

            # For each input batch...
            for i in range(len(true_labels)):
                # The predictions for this batch are a 2-column ndarray (one column for "0" 
                # and one column for "1"). Pick the label with the highest value and turn this
                # in to a list of 0s and 1s.
                pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
                total_target += true_labels[i].tolist()
                total_preds += pred_labels_i.tolist()
                
                # Calculate and store the acc for this batch.
                acc = accuracy_score(true_labels[i], pred_labels_i)          
                acc_set.append(acc)

                f1 = f1_score(true_labels[i], pred_labels_i)
                # print(f"Acc: {acc}\tF1: {f1}")          
                f1_set.append(f1)
                #############################################################################
                # Create a barplot showing the Accuracy score for each batch of test samples.
        avgacc = np.mean(acc_set)
        accuracy_dict[name].append(avgacc)
        avgf1 = np.mean(f1_set)
        f1_dict[name].append(avgf1)
        print(f"AVG Acc: {avgacc}\tAVG F1: {avgf1}")
        scores = precision_recall_fscore_support(total_target, total_preds, average=None, labels=[0,1])
        pr0, pr1 = scores[0]
        re0, re1 = scores[1]
        f10, f11 = scores[2]
        total_acc = accuracy_score(total_target, total_preds)
        total_f1 = f1_score(total_target, total_preds)
        class1_dict_f1[name].append(f11)
        class1_dict_pr[name].append(pr1)
        class1_dict_re[name].append(re1)

        class0_dict_f1[name].append(f10)
        class0_dict_pr[name].append(pr0)
        class0_dict_re[name].append(re0)
        
        # print(f"Total Acc: {total_acc}\tTotal F1: {total_f1}")
        # print(confusion_matrix(total_target, total_preds))
header = "experiments,0,42,23,avg,std\n"
output_acc = header[::]
output_f1 = header[::]
class1_acc = header[::]
class1_f1 = header[::]
class1_pr = header[::]
class1_re = header[::]
class0_acc = header[::]
class0_f1 = header[::]
class0_pr = header[::]
class0_re = header[::]

print("acc")
print(header)
for key in accuracy_dict:
    values = accuracy_dict[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    output_acc += f"{key},{values}\n"
    print(key, ",", values)
print("\n\n")
print("f1")
print(header)
for key in f1_dict:
    values = f1_dict[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    output_f1 += f"{key},{values}\n"
    print(key, ",", values)
print("\n\n")
print("c1 f1")
# print(header)
# for key in class1_dict_acc:
#     values = class1_dict_acc[key]
#     values += [np.mean(values), np.std(values)]
#     values = ",".join([str(elem) for elem in values])
#     class1_acc += f"{key},{values}\n"
#     print(key, ",", values)  
print(header)
for key in class1_dict_f1:
    values = class1_dict_f1[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    class1_f1 += f"{key},{values}\n"
    print(key, ",", values)
print("\n\n")
print("c1 pr")  
print(header)
for key in class1_dict_pr:
    values = class1_dict_pr[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    class1_pr += f"{key},{values}\n"
    print(key, ",", values)
print("\n\n")
print("c1 re")
print(header)
for key in class1_dict_re:
    values = class1_dict_re[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    class1_re += f"{key},{values}\n"
    print(key, ",", values)
print("\n\n")
print("c0 f1")

# print(header)
# for key in class0_dict_acc:
#     values = class0_dict_acc[key]
#     values += [np.mean(values), np.std(values)]
#     values = ",".join([str(elem) for elem in values])
#     class0_acc += f"{key},{values}\n"
#     print(key, ",", values)  
print(header)
for key in class0_dict_f1:
    values = class0_dict_f1[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    class0_f1 += f"{key},{values}\n"
    print(key, ",", values) 
print("\n\n")
print("c0 pr") 
print(header)
for key in class0_dict_pr:
    values = class0_dict_pr[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    class0_pr += f"{key},{values}\n"
    print(key, ",", values)
print("\n\n")
print("c0 re")
print(header)
for key in class0_dict_re:
    values = class0_dict_re[key]
    values += [np.mean(values), np.std(values)]
    values = ",".join([str(elem) for elem in values])
    class0_re += f"{key},{values}\n"
    print(key, ",", values) 
with open("acc_results_5.csv", "w") as fptr:
    fptr.write(output_acc)
    fptr.close()
with open("f1_results_5.csv", "w") as fptr:
    fptr.write(output_f1)
    fptr.close()

# with open("class1_acc_results.csv", "w") as fptr:
#     fptr.write(class1_acc)
#     fptr.close()
with open("class1_f1_results_5.csv", "w") as fptr:
    fptr.write(class1_f1)
    fptr.close()
with open("class1_pr_results_5.csv", "w") as fptr:
    fptr.write(class1_pr)
    fptr.close()
with open("class1_re_results_5.csv", "w") as fptr:
    fptr.write(class1_re)
    fptr.close()

# with open("class0_acc_results.csv", "w") as fptr:
#     fptr.write(class0_acc)
#     fptr.close()
with open("class0_f1_results_5.csv", "w") as fptr:
    fptr.write(class0_f1)
    fptr.close()
with open("class0_pr_results_5.csv", "w") as fptr:
    fptr.write(class0_pr)
    fptr.close()
with open("class0_re_results_5.csv", "w") as fptr:
    fptr.write(class0_re)
    fptr.close()

########## all_errors
fold 1


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

fold 2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

fold 3


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

fold 2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

fold 3


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at